In [1]:
import os
import requests
import gzip
import json
from io import BytesIO

# Function to create a folder if it doesn't exist
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to download, decompress, and save CVE data for a given year
def fetch_and_save_cve_data(year, folder):
    url = f'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-{year}.json.gz'
    response = requests.get(url)

    if response.status_code == 200:
        with gzip.GzipFile(fileobj=BytesIO(response.content)) as f:
            data = json.load(f)

        # Save the fetched data to a JSON file in the specified folder
        filename = os.path.join(folder, f'all_cve_entries_{year}.json')
        with open(filename, 'w') as outfile:
            json.dump(data, outfile, indent=2)
        print(f"CVE data for {year} saved to {filename}")
    else:
        print(f"Failed to retrieve data for {year}. Status code: {response.status_code}")

# Create a folder named 'cve_data' to store all CVE data files
folder_name = 'cve_data'
create_directory(folder_name)

# List of all years for which the CVE data is available
start_year = 2002  # CVE JSON data starts from 2002
end_year = 2024    # Current year
years = range(start_year, end_year + 1)

# Loop through all the years and fetch the data, saving each file in the 'cve_data' folder
for year in years:
    fetch_and_save_cve_data(year,folder_name)

CVE data for 2002 saved to cve_data/all_cve_entries_2002.json
CVE data for 2003 saved to cve_data/all_cve_entries_2003.json
CVE data for 2004 saved to cve_data/all_cve_entries_2004.json
CVE data for 2005 saved to cve_data/all_cve_entries_2005.json
CVE data for 2006 saved to cve_data/all_cve_entries_2006.json
CVE data for 2007 saved to cve_data/all_cve_entries_2007.json
CVE data for 2008 saved to cve_data/all_cve_entries_2008.json
CVE data for 2009 saved to cve_data/all_cve_entries_2009.json
CVE data for 2010 saved to cve_data/all_cve_entries_2010.json
CVE data for 2011 saved to cve_data/all_cve_entries_2011.json
CVE data for 2012 saved to cve_data/all_cve_entries_2012.json
CVE data for 2013 saved to cve_data/all_cve_entries_2013.json
CVE data for 2014 saved to cve_data/all_cve_entries_2014.json
CVE data for 2015 saved to cve_data/all_cve_entries_2015.json
CVE data for 2016 saved to cve_data/all_cve_entries_2016.json
CVE data for 2017 saved to cve_data/all_cve_entries_2017.json
CVE data

In [3]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
import os
import json
import spacy
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load spaCy model for POS tagging, dependency parsing, and NER
nlp = spacy.load('en_core_web_sm')

# Function to extract relevant fields from a CVE item
def extract_cve_fields(cve_item):
    try:
        fields = {
            'CVE_ID': cve_item.get('cve', {}).get('CVE_data_meta', {}).get('ID', 'N/A'),
            'ASSIGNER': cve_item.get('cve', {}).get('CVE_data_meta', {}).get('ASSIGNER', 'N/A'),
            'Description': ' '.join([desc.get('value', '') for desc in cve_item.get('cve', {}).get('description', {}).get('description_data', [])]),
            'Problem_Type': ','.join([ptype.get('value', '') for ptype in cve_item.get('cve', {}).get('problemtype', {}).get('problemtype_data', [])]),
            'Published_Date': cve_item.get('publishedDate', 'N/A'),
            'Last_Modified_Date': cve_item.get('lastModifiedDate', 'N/A'),
            'CVSS_v3_Base_Score': cve_item.get('impact', {}).get('baseMetricV3', {}).get('cvssV3', {}).get('baseScore', 'N/A'),
            'CVSS_v3_Vector_String': cve_item.get('impact', {}).get('baseMetricV3', {}).get('cvssV3', {}).get('vectorString', 'N/A'),
            'Confidentiality_Impact_v3': cve_item.get('impact', {}).get('baseMetricV3', {}).get('cvssV3', {}).get('confidentialityImpact', 'N/A'),
            'Integrity_Impact_v3': cve_item.get('impact', {}).get('baseMetricV3', {}).get('cvssV3', {}).get('integrityImpact', 'N/A'),
            'Availability_Impact_v3': cve_item.get('impact', {}).get('baseMetricV3', {}).get('cvssV3', {}).get('availabilityImpact', 'N/A'),
            'Exploitability_Score_v3': cve_item.get('impact', {}).get('baseMetricV3', {}).get('exploitabilityScore', 'N/A'),
            'Impact_Score_v3': cve_item.get('impact', {}).get('baseMetricV3', {}).get('impactScore', 'N/A'),
            'CVSS_v2_Base_Score': cve_item.get('impact', {}).get('baseMetricV2', {}).get('cvssV2', {}).get('baseScore', 'N/A'),
            'CVSS_v2_Vector_String': cve_item.get('impact', {}).get('baseMetricV2', {}).get('cvssV2', {}).get('vectorString', 'N/A'),
            'Access_Vector_v2': cve_item.get('impact', {}).get('baseMetricV2', {}).get('cvssV2', {}).get('accessVector', 'N/A'),
            'Access_Complexity_v2': cve_item.get('impact', {}).get('baseMetricV2', {}).get('cvssV2', {}).get('accessComplexity', 'N/A'),
            'Authentication_v2': cve_item.get('impact', {}).get('baseMetricV2', {}).get('authentication', 'N/A'),
            'Confidentiality_Impact_v2': cve_item.get('impact', {}).get('baseMetricV2', {}).get('cvssV2', {}).get('confidentialityImpact', 'N/A'),
            'Integrity_Impact_v2': cve_item.get('impact', {}).get('baseMetricV2', {}).get('cvssV2', {}).get('integrityImpact', 'N/A'),
            'Availability_Impact_v2': cve_item.get('impact', {}).get('baseMetricV2', {}).get('cvssV2', {}).get('availabilityImpact', 'N/A'),
            'Exploitability_Score_v2': cve_item.get('impact', {}).get('baseMetricV2', {}).get('exploitabilityScore', 'N/A'),
            'Configurations': [cpe.get('cpe23Uri', '') for node in cve_item.get('configurations', {}).get('nodes', []) for cpe in node.get('cpe_match', [])],
            'Reference_Data': [ref.get('url', '') for ref in cve_item.get('cve', {}).get('references', {}).get('reference_data', [])],
        }

        # Add NLP processing to the description
        doc = nlp(fields['Description'])
        fields['POS_Tags'] = [(token.text, token.pos_) for token in doc]
        fields['NER'] = [(ent.text, ent.label_) for ent in doc.ents]

        return fields
    except Exception as e:
        print(f"Error processing CVE: {cve_item.get('cve', {}).get('CVE_data_meta', {}).get('ID', 'Unknown')} - {str(e)}")
        return None

# Function to process each JSON file in the directory
def process_cve_data_directory(directory):
    all_cve_data = []

    # Loop through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                data = json.load(file)

                # Loop through each CVE entry in the file
                for cve_item in data.get('CVE_Items', []):
                    cve_data = extract_cve_fields(cve_item)
                    if cve_data:
                        all_cve_data.append(cve_data)

    # Convert the collected data into a DataFrame and save as an Excel file
    df = pd.DataFrame(all_cve_data)

    # Save file to your Google Drive
    df.to_excel('/content/drive/My Drive/extracted_cve_data.xlsx', index=False)
    print("CVE data extracted and saved to 'extracted_cve_data.xlsx' in your Google Drive")

# Call the processing function for the 'cve_data' folder
process_cve_data_directory('cve_data')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
